# 🏠 AI Room Redesign Studio - Google Colab

Transform your room images and videos with AI-powered interior design!

**Features:**
- ⚡ GPU-accelerated processing (<1 minute for images)
- 🎨 Multiple styles: Modern, Luxury, Minimal, Custom
- 🖼️ Support for images and videos
- 🌐 Web interface with real-time progress

**Instructions:**
1. Enable GPU: Runtime → Change runtime type → GPU
2. Run all cells in order
3. Upload your room image/video when prompted
4. Get your redesigned room!


## 🚀 Setup & Installation

In [ ]:
# Check GPU availability
import torch
import os

print("🔍 System Information:")
print(f"Python version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Please enable GPU: Runtime → Change runtime type → GPU")

print(f"\n💾 Disk space: {os.statvfs('/').f_bavail * os.statvfs('/').f_frsize / 1e9:.1f} GB available")

In [ ]:
# Clone the repository
!git clone https://github.com/Abbastouqi/ai-room-styling.git
%cd ai-room-styling

print("✅ Repository cloned successfully!")

In [ ]:
# Install dependencies
print("📦 Installing dependencies...")

# Install core ML libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate safetensors
!pip install ultralytics opencv-python pillow numpy
!pip install flask flask-cors werkzeug
!pip install timm huggingface-hub

# Install SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git

print("✅ Dependencies installed!")

In [ ]:
# Download required models
print("🔽 Downloading models...")

import os
import urllib.request
from pathlib import Path

# Create models directory
os.makedirs('models', exist_ok=True)

# Download SAM model
sam_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
sam_path = "models/sam_vit_b.pth"

if not os.path.exists(sam_path):
    print("Downloading SAM model (375MB)...")
    urllib.request.urlretrieve(sam_url, sam_path)
    print("✅ SAM model downloaded")
else:
    print("✅ SAM model already exists")

# YOLOv8 will be downloaded automatically by ultralytics
print("✅ Models ready!")

## 🎨 Usage Methods

Choose one of the methods below:

### Method 1: 🌐 Web Interface (Recommended)

Run the full web application with beautiful UI

In [ ]:
# Method 1: Web Interface with ngrok
!pip install pyngrok

from pyngrok import ngrok
import threading
import time
import sys
sys.path.append('/content/ai-room-styling')

# Set ngrok auth token (optional - get free token from ngrok.com)
# ngrok.set_auth_token("your_token_here")

def start_backend():
    """Start Flask backend"""
    os.chdir('/content/ai-room-styling/backend')
    os.system('python app.py')

# Start backend in background
backend_thread = threading.Thread(target=start_backend, daemon=True)
backend_thread.start()

# Wait for backend to start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(5000)
print(f"🌐 Backend API: {public_url}")

# Serve frontend
os.chdir('/content/ai-room-styling/frontend')
frontend_url = ngrok.connect(8080)
print(f"🎨 Frontend UI: {frontend_url}")
print("\n🚀 Click the Frontend UI link above to access the web interface!")
print("📝 Note: Update the API URL in script.js if needed")

# Start frontend server
!python -m http.server 8080

### Method 2: 📱 Simple Upload Interface

Upload files directly in Colab and process them

In [ ]:
# Method 2: Simple Colab Interface
import sys
sys.path.append('/content/ai-room-styling')

from google.colab import files
import asyncio
import os
from pathlib import Path
from IPython.display import Image, display, HTML
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import numpy as np

# Import our optimized pipeline
from src.optimized_pipeline import OptimizedPipeline, OptimizationConfig

print("🎨 AI Room Redesign - Simple Interface")
print("📁 Upload your room image or video below:")

# Upload files
uploaded = files.upload()

# Process uploaded files
for filename in uploaded.keys():
    print(f"\n📂 Processing: {filename}")
    
    # Save uploaded file
    input_path = f"/content/ai-room-styling/data/input/{filename}"
    os.makedirs(os.path.dirname(input_path), exist_ok=True)
    
    with open(input_path, 'wb') as f:
        f.write(uploaded[filename])
    
    print(f"✅ Saved to: {input_path}")
    
    # Show original image if it's an image
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        print("\n🖼️ Original Image:")
        display(Image(input_path, width=400))

print("\n✅ Upload complete! Run the next cell to process.")

In [ ]:
# Configure processing options
print("⚙️ Configuration:")

# Style selection
style = "modern"  # Options: "modern", "luxury", "minimal", "custom"
custom_prompt = None  # Set this if style="custom"

# If you want custom style, uncomment and modify:
# style = "custom"
# custom_prompt = "Scandinavian living room with light wood furniture, white walls, cozy textiles"

print(f"Style: {style}")
if custom_prompt:
    print(f"Custom prompt: {custom_prompt}")

# Initialize optimized pipeline
config = OptimizationConfig(
    use_gpu=torch.cuda.is_available(),
    batch_size=4 if torch.cuda.is_available() else 2,
    use_fp16=torch.cuda.is_available(),
    cache_models=True,
    parallel_stages=True,
    memory_efficient=True
)

print(f"\n🚀 Initializing pipeline (GPU: {config.use_gpu})...")
pipeline = OptimizedPipeline(config)

# Get input files
input_dir = Path("/content/ai-room-styling/data/input")
input_files = list(input_dir.glob("*"))
input_files = [f for f in input_files if f.suffix.lower() in ['.jpg', '.jpeg', '.png', '.mp4', '.avi', '.mov']]

if not input_files:
    print("❌ No input files found. Please upload files first.")
else:
    print(f"\n📁 Found {len(input_files)} file(s) to process")
    
    # Process files
    print("\n🎨 Starting AI room redesign...")
    print("⏱️ This may take 30-60 seconds with GPU, 5-10 minutes with CPU")
    
    import time
    start_time = time.time()
    
    # Run the pipeline
    results = await pipeline.process_batch(
        input_paths=[str(f) for f in input_files],
        style=style,
        custom_prompt=custom_prompt
    )
    
    processing_time = time.time() - start_time
    
    # Save results
    output_dir = Path("/content/ai-room-styling/data/output")
    pipeline.save_results(results, output_dir)
    
    print(f"\n✅ Processing complete in {processing_time:.1f} seconds!")
    print(f"📁 Results saved to: {output_dir}")
    
    # Display results
    print("\n🎨 Results:")
    
    for result in results:
        input_path = Path(result['input_path'])
        output_files = list(output_dir.glob(f"{input_path.stem}_redesigned.*"))
        
        if output_files:
            output_file = output_files[0]
            
            print(f"\n📸 {input_path.name} → {output_file.name}")
            
            # Show comparison for images
            if input_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
                
                # Original
                original = PILImage.open(input_path)
                ax1.imshow(original)
                ax1.set_title("Original")
                ax1.axis('off')
                
                # Redesigned
                if output_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    redesigned = PILImage.open(output_file)
                    ax2.imshow(redesigned)
                    ax2.set_title(f"Redesigned ({style.title()})")
                    ax2.axis('off')
                
                plt.tight_layout()
                plt.show()
            
            # Download link
            print(f"💾 Download: {output_file}")
    
    # Cleanup
    pipeline.cleanup()
    
    print("\n🎉 All done! You can download the results from the files panel.")

### Method 3: 🔗 Direct API Usage

Use the pipeline directly with Python code

In [ ]:
# Method 3: Direct API Usage
import sys
sys.path.append('/content/ai-room-styling')

import asyncio
import torch
from pathlib import Path
from src.optimized_pipeline import OptimizedPipeline, OptimizationConfig

# Example: Process a sample image
async def process_sample():
    # Configuration
    config = OptimizationConfig(
        use_gpu=torch.cuda.is_available(),
        batch_size=4,
        use_fp16=True,
        cache_models=True
    )
    
    # Initialize pipeline
    pipeline = OptimizedPipeline(config)
    
    # You can replace this with your own image URL or upload
    sample_image_url = "https://images.unsplash.com/photo-1586023492125-27b2c045efd7?w=512&h=512&fit=crop"
    
    # Download sample image
    import urllib.request
    os.makedirs('/content/sample_input', exist_ok=True)
    sample_path = '/content/sample_input/room.jpg'
    
    print("📥 Downloading sample room image...")
    urllib.request.urlretrieve(sample_image_url, sample_path)
    
    # Process with different styles
    styles = ['modern', 'luxury', 'minimal']
    
    for style in styles:
        print(f"\n🎨 Processing with {style} style...")
        
        results = await pipeline.process_batch(
            input_paths=[sample_path],
            style=style
        )
        
        # Save results
        output_dir = Path(f'/content/output_{style}')
        pipeline.save_results(results, output_dir)
        
        print(f"✅ {style.title()} style complete!")
    
    # Cleanup
    pipeline.cleanup()
    
    print("\n🎉 All styles processed! Check the output folders.")

# Run the example
await process_sample()

## 📥 Download Results

Download your processed images/videos

In [ ]:
# Download all results
import zipfile
from google.colab import files
import os

def create_results_zip():
    """Create a zip file with all results"""
    zip_path = '/content/room_redesign_results.zip'
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        # Add files from output directories
        output_dirs = [
            '/content/ai-room-styling/data/output',
            '/content/output_modern',
            '/content/output_luxury', 
            '/content/output_minimal'
        ]
        
        for output_dir in output_dirs:
            if os.path.exists(output_dir):
                for root, dirs, files in os.walk(output_dir):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, '/content')
                        zipf.write(file_path, arcname)
                        print(f"Added: {arcname}")
    
    return zip_path

# Create and download zip
if any(os.path.exists(d) for d in ['/content/ai-room-styling/data/output', '/content/output_modern']):
    print("📦 Creating results archive...")
    zip_path = create_results_zip()
    
    print(f"✅ Results archived: {zip_path}")
    print("💾 Downloading...")
    
    files.download(zip_path)
    print("🎉 Download complete!")
else:
    print("❌ No results found. Please process some images first.")

## 💡 Tips & Troubleshooting

### Performance Tips:
- **Enable GPU**: Runtime → Change runtime type → GPU (T4 recommended)
- **Use smaller images**: 512x512 works best
- **Close other tabs**: Free up GPU memory

### Common Issues:
- **Out of Memory**: Reduce batch_size in config
- **Slow processing**: Make sure GPU is enabled
- **Model download fails**: Check internet connection

### Expected Processing Times:
- **With GPU (T4)**: 30-60 seconds per image
- **With CPU**: 5-10 minutes per image
- **Videos**: 2-5 minutes with GPU, 1-3 hours with CPU

### Supported Formats:
- **Images**: JPG, PNG, JPEG
- **Videos**: MP4, AVI, MOV
- **Max size**: 100MB per file
